In [5]:
from binance.client import Client
import datetime
from pytz import timezone
import os
import pandas as pd
from pandas import Series, DataFrame

In [6]:
client = Client('', '')

In [183]:
datetime.datetime.fromtimestamp(klines[0][0]/1000).strftime('%Y-%m-%d %H:%M:%S KST')

'2018-07-27 13:00:00 KST'

In [109]:
klines[14]

[1532667600000,
 '0.00167860',
 '0.00168940',
 '0.00167650',
 '0.00168210',
 '26905.42000000',
 1532671199999,
 '45.30756854',
 2100,
 '11022.13000000',
 '18.55915285',
 '0']

In [111]:
klines[14]

[1532667600000,
 '0.00167860',
 '0.00168940',
 '0.00167650',
 '0.00168160',
 '27522.32000000',
 1532671199999,
 '46.34475107',
 2153,
 '11309.51000000',
 '19.04259221',
 '0']

In [172]:
now_date = datetime.datetime.today()
now_date - datetime.timedelta(days=365)

datetime.datetime(2017, 7, 27, 15, 34, 51, 576513)

In [184]:
[1,2,3] + [2,4,4]

[1, 2, 3, 2, 4, 4]

In [1]:
HOUR_FORMAT = '%Y/%m/%d %H:%M:%S'

def to_date(date_str) :
    return datetime.datetime.strptime(date_str, HOUR_FORMAT)

def to_hour_str(timestamp) :
    if isinstance(timestamp, pd.Series) :
        return  pd.DatetimeIndex(pd.to_datetime(timestamp, unit='ms')).tz_localize('UTC').tz_convert('Asia/Seoul').strftime(HOUR_FORMAT)
    elif isinstance(timestamp, datetime.datetime) : 
        return timestamp.strftime(HOUR_FORMAT)
    else :
        return datetime.datetime.fromtimestamp(timestamp/1000).strftime(HOUR_FORMAT)

    
def get_klines_binance_data(symbol, start) :
    TZ_STR = ' KST'
    client = Client('', '')
    klines = []
    if start == '' :
        now_datetime = datetime.datetime.today()
        start = to_hour_str(now_datetime - datetime.timedelta(days=365)) 
        end = to_hour_str(now_datetime - datetime.timedelta(hours=1)) 
        print(end)
        while True :
            klines_new = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, start + TZ_STR, end + TZ_STR)
            if len(klines_new) == 0:
                break
            klines = klines_new +  klines   
            end = to_hour_str(to_date(start) - datetime.timedelta(hours=1))  
            start = to_hour_str(to_date(start) - datetime.timedelta(days=365))  
    else :    
        klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, start + TZ_STR)
        klines = klines[:-1]
    
    df_klines = pd.DataFrame(klines, columns=['open-time', 'open', 'high', 'low', 'close', 'volume', 
                                              'close-time', 'quote-volume', 'trades-cnt', 
                                              'taker-base-volume', 'taker-quote-volume', 'unknown'])   
    df_klines = df_klines.assign(hour = to_hour_str(df_klines['open-time']))
    df_klines = df_klines[['hour', 'open', 'high', 'low', 'close', 'volume']]
    return df_klines
    
def get_klines_data(symbol) :
    file_path = './data/' + symbol + '.csv'
    if os.path.isfile(file_path) :
        data = pd.read_csv(file_path)
        data = data[:-1]
        hours_last = data.tail(1)['hour'].to_string(index=False)
        hour_next = to_hour_str(to_date(hours_last) + datetime.timedelta(hours=1))
        new_data = get_klines_binance_data(symbol, hour_next)
        if len(new_data) > 0 :
            data = data.append(new_data, ignore_index=True)
            data.to_csv(file_path, index=False)
    else :
        data = get_klines_binance_data(symbol, '')
        data.to_csv(file_path, index=False)
    return data

# 데이터를 스케일링 시킨다.
def get_scaled_data(data) :
    scaled_data = data.copy()
    scaled_data = scaled_data[['close', 'open', 'high', 'low', 'volume']]
    scaled_data = scaled_data[scaled_data['close']!=0]
    scaled_data['close'], scaler_close = get_scaled_cols(scaled_data, 'close')
    scaled_data['open'], _ = get_scaled_cols(scaled_data, 'open')
    scaled_data['high'], _ = get_scaled_cols(scaled_data, 'high')
    scaled_data['low'], _ = get_scaled_cols(scaled_data, 'low')
    scaled_data['volume'], _ = get_scaled_cols(scaled_data, 'volume')
    return scaled_data, scaler_close;


data = get_klines_data('BNBBTC')
data

NameError: name 'os' is not defined

In [47]:
get_klines_binance_data('BNBBTC', '2018/08/07 11:00:00 KST')

,hour,open,high,low,close,volume
0,2018/08/07 02:00:00,0.00196180,0.00196770,0.00194540,0.00195340,59240.94000000


In [210]:
to_date('2018/07/27 15:00:00 KST')

datetime.datetime(2018, 7, 27, 15, 0)